In [67]:
import tensorflow as tf
from tensorflow.keras.models import Model, Input
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, concatenate
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [ ]:
#Import the data
input_path = "./CAT_Infrasound_Data/model_dataset1/input.json"
labels_path = "./CAT_Infrasound_Data/model_dataset1/labels.json"

data = (pd.read_json(input_path)).values.tolist()

reshapedData = []
fftLength = int(np.sqrt(len(data[0][1])))

for element in data:
    atmoStats = np.array(element[0])
    fft = np.array(element[1]).reshape(fftLength, fftLength, 1)
    reshapedData.append([atmoStats, fft])

labels = (pd.read_json(labels_path)).values.tolist()

#Split into train, val, test
data_train, data_temp, labels_train, labels_temp = train_test_split(data, labels, test_size=0.3)
data_val, data_test, labels_val, labels_test = train_test_split(data_temp, labels_temp, test_size=0.5, random_state=42)

In [ ]:
#Model Design

#atmostats input model
atmoInput = Input(shape = (len(data[0][0]), 1))
x = atmoInput

numLayersAtmo = 10
for i in range(0, numLayersAtmo):
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)

atmostatsOutput = Dense(256, activation = 'relu')(x)

#fft input model
fftInput = Input(shape = (fftLength, fftLength, 1))
y = fftInput

numLayersFFT = 5
for i in range(0, numLayersFFT):
    y = Conv2D(32, kernel_size=4, activation='relu')(y)
    y = MaxPooling2D(pool_size=(2, 2))(y)

fftOutput = Flatten()(y)

#concatenate, feed through another DNN
concatenatedInput = concatenate([atmostatsOutput, fftOutput])

numLayersFinal = 40
for i in range(0, numLayersAtmo):
    z = Dense(512, activation='relu')(z)
    z = Dropout(0.2)(z)

z = Dense(256, activation='relu')(z)
z = Dropout(0.2)(z)

z = Dense(128, activation='relu')(z)
z = Dropout(0.2)(z)

z = Dense(64, activation='relu')(z)
z = Dropout(0.2)(z)

z = Dense(32, activation='relu')(z)
z = Dropout(0.2)(z)

z = Dense(16, activation='relu')(z)

z = Dense(8, activation='relu')(z)

z = Dense(4, activation='relu')(z)

#final output layer
output = Dense(2, activation = 'relu')(z)

#model compilation
model = Model(inputs=[atmoInput, fftInput], outputs=output)

opt = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=opt, loss='mean_squared_error', metrics=['mean_squared_error'])

In [ ]:
#model training
size = 32
numEpochs = 100
model.fit(data_train, labels_train, validation_data = (data_val, labels_val), batch_size = , epochs = 100)